In [ ]:
import numpy as np
import random
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# let's make some data

In [ ]:
p = 1.0
while p>0.05:
    a = np.random.normal(50.,7.4,size=100)
    b = np.random.normal(53.5,15.9,size=200)
    t,p = stats.ttest_ind(a,b,equal_var=False,)
fig,ax = plt.subplots()
ax.hist(a,color='blue',alpha=0.1)
ax.hist(b,color='red',alpha=0.1)
ax.axvspan(a.mean()-a.std()/np.sqrt(a.size),a.mean()+a.std()/np.sqrt(a.size),color='blue',alpha=0.7,label='sample a')
ax.axvspan(b.mean()-b.std()/np.sqrt(b.size),b.mean()+b.std()/np.sqrt(b.size),color='red', alpha=0.7,label='sample b')
ax.legend()

## parametric independent t test with equal variance assumption

In [ ]:
stats.ttest_ind(a,b,equal_var=True,)

## parametric independent t test without equal variance assumption

In [ ]:
stats.ttest_ind(a,b,equal_var=False,)

## Wilcoxon rank-sum statistics for two samples

In [ ]:
stats.ranksums(a,b,)

In [ ]:
# Wilcoxon-Mann-Whitney statistics
stats.mannwhitneyu(a,b)

## nonparametric t test by permutation test

In [ ]:
fig, ax = plt.subplots()
difference = np.mean(a) - np.mean(b)
results=[]
for ii in range(1000):
    c = np.concatenate([a,b])
    random.shuffle(c)
    new_a = c[:100]
    new_b = c[100:]
    new_diff = new_a.mean() - new_b.mean()
    results.append(new_diff)
results = np.array(results)
ax.hist(results,color='blue',alpha=0.6,bins=20)
p = min([stats.percentileofscore(results,difference,)/100,1-stats.percentileofscore(results,difference,)/100])
ax.axvline(difference,color='red',label='sample differences\np value: %.2f'%p)
ax.legend()

## redo the permutation test multiple times and sample the distribution of the p values

In [ ]:
results = []
for ii in range(50):
    temp=[]
    for jj in range(1000):
        c = np.concatenate([a,b])
        random.shuffle(c)
        new_a = c[:100]
        new_b = c[100:]
        new_diff = new_a.mean() - new_b.mean()
        temp.append(new_diff)
    temp = np.array(temp)
    p=min([stats.percentileofscore(temp,difference,)/100,1-stats.percentileofscore(temp,difference,)/100])
    results.append(p)
fig, ax = plt.subplots()
ax.hist(results,bins=10,label='p values: %.3f+/-%.3f'%(np.mean(results),np.std(results)))
ax.legend(loc='best')

# Bayesian stats

In [ ]:
import pymc3 as pm

In [ ]:
c = np.concatenate([a,b])
c_m = c.mean()
c_s = c.std()*2
with pm.Model() as model:
    group1_mean = pm.Normal('group1_mean', c_m, sd=c_s)
    group2_mean = pm.Normal('group2_mean', c_m, sd=c_s)

In [ ]:
σ_low = 6
σ_high = 16

with model:
    group1_std = pm.Uniform('group1_std', lower=σ_low, upper=σ_high)
    group2_std = pm.Uniform('group2_std', lower=σ_low, upper=σ_high)

In [ ]:
with model:
    v = pm.Exponential('v_minus_one',1/29.) +1

In [ ]:
with model:
    λ1 = group1_std**-2
    λ2 = group2_std**-2

    group1 = pm.StudentT('drug', nu=v, mu=group1_mean, lam=λ1, 
                         observed=a)
    group2 = pm.StudentT('placebo', nu=v, mu=group2_mean, lam=λ2, 
                         observed=b)

In [ ]:
with model:
    diff_of_means = pm.Deterministic('difference of means', 
                                     group1_mean - group2_mean)
    diff_of_stds = pm.Deterministic('difference of stds', 
                                    group1_std - group2_std)
    effect_size = pm.Deterministic('effect size',
                                   diff_of_means / np.sqrt((group1_std**2 + group2_std**2) / 2))

In [ ]:
with model:
    trace = pm.sample(2000,init=None,njobs=1)

In [ ]:
_=pm.plot_posterior(trace[100:],
                  varnames=['group1_mean', 
                            'group2_mean', 
                            'group1_std', 
                            'group2_std', 
                            'v_minus_one'],
                  color='#87ceeb');

In [ ]:
_=pm.plot_posterior(trace[1000:],
                  varnames=['difference of means', 'difference of stds', 'effect size'],
                  ref_val=0,
                  color='#87ceeb')

In [ ]:
_=pm.forestplot(trace[1000:], 
                varnames=[v.name for v in model.vars[:2]],
           )

In [ ]:
_=pm.forestplot(trace[1000:], varnames=[v.name for v in model.vars[2:]])

# permutation test with classification

In [ ]:
from sklearn.model_selection import StratifiedKFold,permutation_test_score
from sklearn.svm import SVC

In [ ]:
label = np.concatenate([np.zeros(a.size),np.ones(b.size)])
X = np.concatenate([a,b])
#print(X.shape,label.shape)
cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=12345)
clf = SVC(class_weight='balanced',random_state=12345)
score, permutation_scores, sig = permutation_test_score(clf,X.reshape(-1,1),label,cv=cv,random_state=12345,n_permutations=300,
                                                       scoring='roc_auc')

fig, ax = plt.subplots(figsize=(8,4))
ax.hist(permutation_scores,bins=50,label='randomized')
ax.axvline(np.mean(permutation_scores),color='blue',lw=2,label='chance level')
ax.axvline(score,color='red',lw=2,label='classification score')
ax.legend()
ax.set(title='support vector machine',xlabel='scoring metric',ylabel='Count')
print('sig: %.4f'%sig)

# maybe we should try some other estimators to see whether we will have the same results

In [ ]:
names = ['random forest','logistic regression', 'SVM linear kernel','KNN-default','extra tree','gaussian process']
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import ExtraTreeClassifier

In [ ]:
clfs = [RandomForestClassifier(n_estimators=50,class_weight='balanced',random_state=12345),
       LogisticRegression(class_weight='balanced',random_state=12345),
       SVC(kernel='linear',class_weight='balanced',random_state=12345),
       KNeighborsClassifier(),
       ExtraTreeClassifier(class_weight='balanced',random_state=12345),
       GaussianProcessClassifier(random_state=12345)]

In [ ]:
fig, axes = plt.subplots(figsize=(16,8),nrows=2,ncols=3)
for name,clf,ax in zip(names,clfs,axes.flatten()):
    score, permutation_scores, sig = permutation_test_score(clf,X.reshape(-1,1),label,cv=cv,random_state=12345,n_permutations=300,
                                                       scoring='roc_auc')
    ax.hist(permutation_scores,bins=50,label='randomized')
    ax.axvline(np.mean(permutation_scores),color='blue',lw=2,label='chance level:%.2f'%np.mean(permutation_scores))
    ax.axvline(score,color='red',lw=2,label='clf.score:%.2f'%score)
    ax.legend(loc='best')
    ax.set(title=name)
    print('finish %s,%.4f'%(name,sig))